## Report No.02 二分法とNLsolveパッケージ
### sin(x) = 3/4 の近似解を求めてみよう
- 二分法を用いて自分でプログラムして近似解を求める

In [1]:
f(x)=sin(x)-3/4      #関数を定義
left, right = 0.0, 1.0

(0.0, 1.0)

In [2]:
f(left)*f(right)     #定義域内にf(x)=0なるxが存在することを確認

-0.06860323860592238

In [3]:
sq = [left right]
for i in 0:100           #二分法を用いて近似解を求める
    av = (left+right)/2
    
    if abs(f(av)) < 1.0e-8
        println(i,":",av)
        break
    end
    
    if f(av)*f(left) < 0
        right = av
    else
        left = av
        end 
    sq = vcat(sq, [left right])
end

24:0.8480620682239532


以上から、近似解はx=0.8480620682239532と求まる。

---

- NLsolveパッケージを用いて近似解を求める

In [4]:
using NLsolve

function nls(func, params...; ini = [0.0])
    if typeof(ini) <: Number
        r = nlsolve((vout,vin)->vout[1]=func(vin[1],params...), [ini])
        v = r.zero[1]
    else
        r = nlsolve((vout,vin)->vout .= func(vin,params...), ini)
        v = r.zero
    end
    return v, r.f_converged
end

nls(f, ini = 0.9)

(0.8480620789800516, true)

以上から、NLsolveパッケージを使って求めた近似解は0.848062078981481となる。

---

### 非線形連立方程式の近似解も求めてみよう

In [5]:
function f(vx)       #関数を定義
    x, y, z = vx
    return [
        x+2y+z+1
        x^2+2y^2+z^2-10
        sin(x+y+z)-0.7
    ]
end

f (generic function with 1 method)

In [6]:
nls(f, ini = [1.0, 0.0, 1.0])

([0.804946, -1.7754, 1.74585], true)

上記の実行結果がNLsolveによって得られた解である。  

---

次に、Newton法を自力でプログラムして近似解を求める。

In [7]:
function J(vx)      #ヤコビアンを定義
    x,y,z = vx
    return [
        1 2 1
        2x 4y 2z
        cos(x+y+z) cos(x+y+z) cos(x+y+z)
    ]
end

J (generic function with 1 method)

In [8]:
vx = [ 1.0, -1.5, 1.5 ]

for i in 1:100            #Newton法を実装する
    if norm(f(vx)) < 1.0e-06
        println(i, ": ", vx,", ", f(vx))
        break
    end
    vx = vx - J(vx)\f(vx)   
end

7: [0.804946, -1.7754, 1.74585], [-2.22045e-16, 1.04095e-12, -1.11022e-16]


上記の実行結果が自作Newton法により求めた近似解である。  
NLsolveパッケージにより求めた解と全く同じ解が得られた。